# Practical work - Mixing models

Useful external references :

> - [NumPy documentation](https://docs.scipy.org/doc/numpy/user/index.html)  
- Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- Documentation MatPlotLib](http://matplotlib.org/)  
- Scikit-learn website](http://scikit-learn.org/stable/index.html)  
- Site langage python](https://www.python.org)  



**The aim** of this tutorial is to introduce the use of Scikit-learn's features for estimating mixture models,
and to contribute to a better understanding of this method and of techniques for choosing the number of components.
To this end, we first examine data generated in a controlled way, and then real data.


## Gaussian mixture models


A presentation of the tools available for mixture models can be found in [http://scikit-learn.org/stable/modules/mixture.html](http://scikit-learn.org/stable/modules/mixture.html).


Model selection (the choice of the number of mixture components, but also the type of covariance matrix between `'full'`, `'tied'`, `'diag'`, `'spherical'`, see below) can be made by calculating *Akaike information criterion* (AIC) or *Bayes information criterion* (BIC).


Among the parameters, let's focus on the following (see [implementation description](http://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html) for the others):


- `n_components`: the number of components in the mixture (`1` by default).  
- `covariance_type` : type of covariance matrix used, choice between `'full'`, `'tied'`, `'diag'` and `'spherical'`, default `'full'` ; `'full'` = any matrix, `'tied'` = any matrix but identical between the different components of the mixture, `'diag'` = diagonal matrix (any variances but zero covariances), `'spherical'` = each component has its own variance (but this value is shared by all variables for that component) and zero covariances.  
- `init_params`: parameter initialization method, choice between `'kmeans'` (automatic classification of observations with *K-means*, then use of each group center) and `'kmeans'` (automatic classification of observations with *K-means*, then use of each group center).
- `n_init`: number of initializations (followed by EM executions) performed; the best result is retained.  




Among the accessible attributes we can mention (see the documentation for the full list):


- `weights_`: the mixing coefficients (weights of the different components of the mixture).  
- `Means_`: the averages of the components.  
- `covariances_`: the covariance matrices of the components.  
- `converged_` : `True` if EM has converged (in `.fit()`), `False` otherwise.  
- `lower_bound_`: log-likelihood achieved at the end of iterations by EM's best execution.  


- `fit(X, y=None)`: calculation of the model from the observations which are the lines of X.  
- aic(X)`: value of the Akaike information criterion on the data of X for the current model.  
- `bic(X)`: value of the Bayes information criterion on the data from X for the current model.  
- `predict_proba(X)`: *a posteriori* probabilities with respect to each component of the current model for each item of X data.  
- `predict(X, y=None)`: group labels (automatic classification from the mixture model using *a posteriori* probabilities) obtained with the current model for $X$ data.  
- `sample([n_samples, random_state])`: generate a sample (composed of `n_samples` data) from the model (for the moment, accessible only for kernels

Methods that can be used:

- `fit(X, y=None)`: calculation of the model from the observations which are the lines of X.  
- aic(X)`: value of the Akaike information criterion on the data of X for the current model.  
- `bic(X)`: value of the Bayes information criterion on the data from X for the current model.  
- `predict_proba(X)`: *a posteriori* probabilities with respect to each component of the current model for each item of X data.
- `predict(X, y=None)`: group labels (automatic classification from the mixture model using *a posteriori* probabilities) obtained with the current model for X data.  
- `sample([n_samples, random_state])`: generate a sample (composed of `n_samples` data) from the model (currently only available for `'gaussian'` and `'tophat'` kernels).  
- `score(X, y=None)`: returns the total log-likelihood of X data with respect to the model.  
- `score_samples(X)`: returns the logarithm of the density calculated for each X data item.  
- `get_params([deep])`: read the parameter values of the estimator used.  
- `set_params(**params)`: give values to the parameters of the estimator used.



### Estimation from generated data

A first use on one-dimensional data (note that the `GaussianMixture` class is available from Scikit-learn version 0.18 onwards; for earlier versions, use the `GMM` class instead):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm
from sklearn.mixture import GaussianMixture

In [ ]:
# sample with  \pi_1=0.3 and \pi_2=0.7 
N = 100
X = np.concatenate((np.random.normal(0, 1, int(0.3 * N)),
                    np.random.normal(5, 1, int(0.7 * N))))[:, np.newaxis]

# prepare the data
X_plot = np.linspace(-5, 10, 1000)[:, np.newaxis]

true_density = (0.3*norm(0,1).pdf(X_plot[:,0]) + 0.7*norm(5,1).pdf(X_plot[:,0]))

# estimation using the correct number of component
gmm = GaussianMixture(n_components=2,n_init=3).fit(X)
print(gmm.converged_)
print(gmm.lower_bound_)

# compute the density
density = np.exp(gmm.score_samples(X_plot))

# Plot true density and estimated density
fig, ax = plt.subplots(1,1,figsize=(12,6))
ax.fill_between(X_plot[:,0], true_density, fc='b', alpha=0.2, label='Vraie densité')
ax.plot(X_plot[:,0], density, '-', label="Estimation")
ax.plot(X[:, 0], -0.005 - 0.01 * np.random.random(X.shape[0]), '+k')
ax.legend(loc='upper left')

plt.show()

### Todo:

Vary the number of components and visually examine the results. Look at the component weights and averages. Examine how the final value reached by the log-likelihood evolves with the number of components.

Next, we test on two-dimensional data :

In [ ]:
# Sample three clusters 
md1 = 1.5 * np.random.randn(200,2) + [3,3]
md2 = np.random.randn(100,2).dot([[2, 0],[0, 0.8]]) + [-3, 3]
md3 = np.random.randn(100,2) + [3, -3]
md = np.concatenate((md1, md2, md3))

# data for density
grid_size = 100
Gx = np.arange(-10, 10, 20/grid_size)
Gy = np.arange(-10, 10, 20/grid_size)
Gx, Gy = np.meshgrid(Gx, Gy)

# estimation of the gmm
gmm = GaussianMixture(n_components=3,n_init=3).fit(md)

# compute true model
density = np.exp(gmm.score_samples(np.hstack(((Gx.reshape(grid_size*grid_size))[:,np.newaxis],
        (Gy.reshape(grid_size*grid_size)[:,np.newaxis])))))

# show 3d results
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(16,8))
ax.plot_surface(Gx, Gy, density.reshape(grid_size,grid_size), rstride=1,
                    cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=True)
ax.scatter(md[:,0], md[:,1], -0.025)

plt.show()

### Todo:

Vary the number of components and examine the results visually. See how the final log-likelihood value changes with the number of components.

### Todo:

Generate two-dimensional data following a **uniform** distribution in $[0, 1]^2 $ (two-dimensional data in the unit square). Estimate a Gaussian mixture with 3 components, using `n_init = 1`. Visualize the results. Use the `predict` method of `GaussianMixture` to obtain group labels for the data. Apply the modeling followed by group label assignment several times in succession and examine the **stability of the partitionings** using [the adjusted Rand index](http://scikit-learn.org/stable/modules/clustering.html#adjusted-rand-index), as in automatic classification practical exercises.

Note that you don't have any groups defined at the outset; to define reference groups, to which you'll compare those derived from other classifications, you can first apply density estimation () followed by group label assignment with `predict`. What do you see?

### Choosing the number of components and type of covariance matrix for the generated data

To choose the number of components in the mixture, we first compare the AIC and BIC criteria, using `'full'` (default) covariance matrices. On generated two-dimensional data :

In [ ]:
n_max = 8    # maximal number of components
n_components_range = np.arange(n_max)+1
aic = []
bic = []

# estimate models and get aic and bic values
for n_comp in n_components_range:
    gmm = GaussianMixture(n_components=n_comp).fit(md)
    aic.append(gmm.aic(md))
    bic.append(gmm.bic(md))

print("AIC : " + str(aic))
print("BIC : " + str(bic))

# normalization
raic = aic/np.max(aic)
rbic = bic/np.max(bic)

# bar plots
xpos = np.arange(n_max)+1  # bar localization
larg = 0.3              # largeur des barres
fig,ax = plt.subplots(1,1,figsize=(16,8))
ax.set_ylim([min(np.concatenate((rbic,raic)))-0.1, 1.1])
ax.set_xlabel('Nombre de composantes')
ax.set_ylabel('Score')
ax.bar(xpos, raic, larg, color='r', label="AIC")
ax.bar(xpos+larg, rbic, larg, color='b', label="BIC")
ax.legend(loc='upper left')

plt.show()

For these data and with `'full'` covariance matrices, both AIC and BIC favor the use of 3 components, a number equal to that of the normal distributions used to generate the data.

### Todo:

Perform the same comparison for these data with `'diag'` covariance matrices.

### Todo:

Add on the same (adapted) graph the normalized values of the final log-likelihood for each value of `n_components`.

### Estimation from texture data

We will apply Gaussian mixture density estimation to texture data projected on the first two principal axes. These data correspond to 5,500 observations described by 40 variables. Each observation belongs to one of 11 texture classes; each class is represented by 500 observations.
The data are taken from https://sci2s.ugr.es/keel/dataset.php?cod=72

The data can be searched on the Moodle server.

The data are then analyzed:

In [ ]:
# apply PCA
from sklearn.decomposition import PCA
textures = np.loadtxt('./input/texture.dat')
pca = PCA().fit(textures[:,:40])
texturesp = pca.transform(textures[:,:40])

# build gmm on pca components
gmm = GaussianMixture(n_components=11,n_init=3).fit(texturesp[:,:2])
print(gmm.converged_)

# True
print(gmm.n_iter_)
# 10
print(gmm.lower_bound_)
# -1.7780023505669722

# data for the density
grid_size = 100
xmin = 1.3*np.min(texturesp[:,0])
xmax = 1.3*np.max(texturesp[:,0])
Gx = np.arange(xmin, xmax, (xmax-xmin)/grid_size)
ymin = 1.3*np.min(texturesp[:,1])
ymax = 1.3*np.max(texturesp[:,1])
Gy = np.arange(ymin, ymax, (ymax-ymin)/grid_size)
Gx, Gy = np.meshgrid(Gx, Gy)

# compute density
density = np.exp(gmm.score_samples(np.hstack(((Gx.reshape(grid_size*grid_size))[:,np.newaxis],
                                   (Gy.reshape(grid_size*grid_size)[:,np.newaxis])))))

# plots
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(16, 8))
ax.plot_surface(Gx, Gy, density.reshape(grid_size,grid_size), rstride=1,
                    cstride=1, cmap=cm.coolwarm, linewidth=0, antialiased=True)
ax.scatter(texturesp[:,0], texturesp[:,1], -0.25)
plt.show()

### Todo:

Apply Gaussian mixture density estimation to "texture" data projected on the first two **discriminant** axes (obtianed by LDA). Increment the number of discriminant axes.

### Choice of number of components and type of covariance matrix for texture data

First, let's determine the best number of components using the AIC and BIC criteria for projections of these data onto the first two principal axes:

In [ ]:
n_max = 10    # maximal number of components
n_components_range = np.arange(n_max)+6
aic = []
bic = []

# build models and get critria
for n_comp in n_components_range:
    gmm = GaussianMixture(n_components=n_comp).fit(texturesp[:,:2])
    aic.append(gmm.aic(texturesp[:,:2]))
    bic.append(gmm.bic(texturesp[:,:2]))


# normalize results
raic = aic/np.max(aic)
rbic = bic/np.max(bic)

# bar plot
xpos = n_components_range  # bar localization
larg = 0.3
fig = plt.figure()
plt.ylim([min(np.concatenate((rbic,raic)))-0.02, 1.01])
plt.bar(xpos, raic, larg, color='r', label="AIC")
plt.bar(xpos+larg, rbic, larg, color='b', label="BIC")
plt.legend(loc='upper left')
plt.show()

### Question:

How do you explain that the optimal value obtained with BIC for `n_components` is less than the number of texture classes (which is 11)?

### Question:

Carry out the same study using instead the projections of the "texture" data on the first two **discriminating** axes. How do you explain the difference compared to the previous case?